
# README
# Système QA avec TALN
KAROUIA Alaedine 

<br/>
<br/>
<br/>

# Pré-requis
Le programme nécessite d'importer dans le dossier content de google colab les fichiers relation et question.

En cas de message d'erreur généré par NLTK, merci d'installer les dépendances requises.

<br/>

# Problèmes rencontrés
Lorsque les requêtes sparql ne fonctionne pas, la probabilité d'avoir une réponse correcte se reduit, car on dépend de la requête lookup qui est moins précise. Et il arrive que l'api lookup de ne fonctionner pas. Dans ces cas-là, je ne compte pas l'erreur.


Un autre probleme rencontré est le choix des methodes pour trouver des similarités entre les mots, car ils existent beaucoup de manière de faire.



# Amelioration

Utiliser des corpus de données, cela permettrait de retrouver des mots plus facilement. Les synonymes se trouveraient beaucoup plus facilement de cette manière.
<br/>

# Sources
- Documentation Python
- Documentation NLTK
- https://www.nltk.org/howto/wordnet.html
- http://www.nltk.org/howto/metrics.html
- https://stackoverflow.com/questions/3282823/get-the-key-corresponding-to-the-minimum-value-within-a-dictionary


In [ ]:
pip install sparqlwrapper

#LES IMPORTS

In [ ]:
import nltk
import numpy
from nltk.chunk import conlltags2tree, tree2conlltags
from pprint import pprint
from nltk.tokenize import word_tokenize
from nltk.tag import pos_tag
from nltk.corpus import stopwords
import xml.etree.ElementTree as ET
import random
import spacy
from spacy import displacy
from collections import Counter
import en_core_web_sm
import re
import requests
import asyncio
from nltk.corpus import wordnet
from nltk.corpus import wordnet as wn
from SPARQLWrapper import *

nlp = en_core_web_sm.load()
nltk.download('averaged_perceptron_tagger')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download("stopwords")
#Let install sparqlwrapper which a python wrapper around sparql client




[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

question et reponse de test

In [ ]:
questionsTest = [
    "Which river does the Brooklyn Bridge cross?",
    "Who created Wikipedia?",
    "In which country does the Nile start?",
    "What is the highest place of Karakoram?",
    "Who designed the Brooklyn Bridge?",
    "Who created Goofy?",
    "Who is the mayor of New York City?",
    "Through which countries does the Yenisei river flow?",
    "Which museum exhibits The Scream by Munch?",
    "Which states border Illinois?",
    "Who was the wife of US president Lincoln?",
    "In which programming language is GIMP written?",
    "In which country is the Limerick Lake?",
    "What are the official languages of the Philippines?",
    "Who owns Aldi?",
    "What is the area code of Berlin?",
    "When was the Battle of Gettysburg?",
    "Who developed the video game World of Warcraft?",
    "Give me the homepage of Forbes.",
    "Which awards did WikiLeaks win?",
    "Give me all actors starring in Last Action Hero.",
    "Who is the owner of Universal Studios?",
    "What did Bruce Carver die from?"

]

answersTest = [
    ["http://dbpedia.org/resource/East_River"],
    ["http://dbpedia.org/resource/Jimmy_Wales", "http://dbpedia.org/resource/Larry_Sanger"],
    ["http://dbpedia.org/resource/Ethiopia", "http://dbpedia.org/resource/Ethiopia"],
    ["http://dbpedia.org/resource/K2"],
    ["http://dbpedia.org/resource/John_A._Roebling", "http://dbpedia.org/resource/John_Augustus_Roebling"],
    ["http://dbpedia.org/resource/Art_Babbitt"],
    ["http://dbpedia.org/resource/Michael_Bloomberg"],
    ["http://dbpedia.org/resource/Mongolia", "http://dbpedia.org/resource/Russia"],
    ["http://dbpedia.org/resource/National_Gallery_of_Norway", "http://dbpedia.org/resource/National_Gallery,_Oslo"],
    ["http://dbpedia.org/resource/Kentucky", "http://dbpedia.org/resource/Missouri",
     "http://dbpedia.org/resource/Wisconsin", "http://dbpedia.org/resource/Indiana",
     "http://dbpedia.org/resource/Iowa"],
    ["http://dbpedia.org/resource/Mary_Todd_Lincoln"],
    ["http://dbpedia.org/resource/C_(programming_language)", "http://dbpedia.org/resource/GTK+"],
    ["http://dbpedia.org/resource/Canada"],
    ["http://dbpedia.org/resource/Filipino_language"],
    
    ["http://dbpedia.org/resource/Karl_Albrecht", "http://dbpedia.org/resource/Theo_Albrecht"],
    ["030"],
   
    ["1863-07-03"],
    ["http://dbpedia.org/resource/Blizzard_Entertainment"],
     
    ["http://www.forbes.com"],
    ["http://dbpedia.org/resource/Index_on_Censorship",
     "http://dbpedia.org/resource/Amnesty_International_UK_Media_Awards",
     "http://dbpedia.org/resource/Sam_Adams_Award"],
    ["http://dbpedia.org/resource/Arnold_Schwarzenegger", "http://dbpedia.org/resource/Anthony_Quinn",
     "http://dbpedia.org/resource/F._Murray_Abraham", "http://dbpedia.org/resource/Art_Carney",
     "http://dbpedia.org/resource/Austin_O'Brien", "http://dbpedia.org/resource/Tom_Noonan",
     "http://dbpedia.org/resource/Bridgette_Wilson", "http://dbpedia.org/resource/Charles_Dance",
     "http://dbpedia.org/resource/Robert_Prosky"],
    ["http://dbpedia.org/resource/General_Electric", "http://dbpedia.org/resource/MCA_Inc.",
     "http://dbpedia.org/resource/Seagram", "http://dbpedia.org/resource/Comcast",
     "http://dbpedia.org/resource/NBCUniversal", "http://dbpedia.org/resource/Vivendi",
     "http://dbpedia.org/resource/Independent_business"],
    ["http://dbpedia.org/resource/Cancer"]

]

In [ ]:

tree = ET.parse("/content/questions.xml") # ajouter les fichiers question et relations dans le dossier content de google colab
root = tree.getroot()
gold_standard_answers = []
questions = []
for child in root:
    for question in child:
        if(question.tag == 'string' and question.attrib['lang'] == 'en'):
            questions.append(question.text)
        for answer in question:
              
              for uri in answer:
                  gold_standard_answers.append(uri.text)

print(len(questions))
print(gold_standard_answers)
#on recupere les questions et reponses


25
['http://dbpedia.org/resource/East_River', 'http://dbpedia.org/resource/Jimmy_Wales', 'http://dbpedia.org/resource/Larry_Sanger', 'http://dbpedia.org/resource/Ethiopia', 'http://dbpedia.org/resource/Lake_Tana', 'http://dbpedia.org/resource/K2', 'http://dbpedia.org/resource/John_A._Roebling', 'http://dbpedia.org/resource/The_Walt_Disney_Company', 'http://dbpedia.org/resource/Bob_Ogle', 'http://dbpedia.org/resource/Paul_Murry', '"Bill de Blasio"@en', 'http://dbpedia.org/resource/Ka-Hem', '"National Gallery and Munch Museum"@en', 'http://dbpedia.org/resource/Kentucky', 'http://dbpedia.org/resource/Missouri', 'http://dbpedia.org/resource/Wisconsin', 'http://dbpedia.org/resource/Indiana', 'http://dbpedia.org/resource/Iowa', 'http://dbpedia.org/resource/Mary_Todd_Lincoln', 'http://dbpedia.org/resource/C_(programming_language)', 'http://dbpedia.org/resource/GTK+', 'http://dbpedia.org/resource/Canada', 'http://dbpedia.org/resource/Czech_koruna', 'http://dbpedia.org/resource/Blizzard_Enterta

Preprocess permet de tokeniser une phrase

In [ ]:
def preprocess(sent):
    sent = sent.lower()
    sent = nltk.word_tokenize(sent)
    sent = nltk.pos_tag(sent)
    return sent


preprocess("Give me the homepage of Forbes.")


[('give', 'VB'),
 ('me', 'PRP'),
 ('the', 'DT'),
 ('homepage', 'NN'),
 ('of', 'IN'),
 ('forbes', 'NNS'),
 ('.', '.')]

A l'aide d'un text tokenisé, remove_stop_word va permettre de preparer la question en y enlevant les stop words car cela n'importe pas dans le traitement ensuite.

In [ ]:
def remove_stop_word(tokenize_text):
    stop_words = stopwords.words('english')
    stop_words.append('?')
    stop_words.append('.')
    filtered_sentence = [w for w in tokenize_text if not w in stop_words]
    return filtered_sentence



la fonction get_named_entity permet de recuperer l'entite de la question:
si on ne trouve pas, on prend les noms en majuscule qui sont present dans la phrase.

In [ ]:

def get_named_entity(text):
  nlp = en_core_web_sm.load()
  text = text.replace('?','')
  text = text.replace('.','')
  doc = nlp(text)
  
  named_entity = []

  for ent in doc.ents:
      ne = ent.text
      ne = ne.replace('the ', '')
      ne = ne.replace('The ', '')
      named_entity.append(ne)

  if(named_entity == []):
    sp = text.split()
    res = ""
    for i in range(len(sp)):
      if(i==0):
        pass
      elif sp[i][0].isupper():
        named_entity.append(sp[i])
    for i in range(len(named_entity)):
      if(i == 0):
        res =  named_entity[i]
      else: 
        res = res + " "+ named_entity[i]
    return [res]


  return named_entity


print(get_named_entity("Give me the homepage of Forbes?"))

def switch_question_type(question):#permet de savoir le type de question
  questionL = question.lower()

  res = ''
  nn = []
  
  for i in preprocess(questionL):
    if(i[1][0] == 'W'):
      res =  i[0]
    elif(i[1] == 'NN' or i[1] == 'NNS'):
      nn.append(i[0])

  dico = {  # le dico qui permet de connaitre le type de question
        "what": ["place", "Organisation"],
        "where": ["Place"],
        "why": ["Resource"],# Prendre le NP qui vient après
        "who": ["Person", "Organisation"],
        "whose": ["Resource"],# Prendre le NP qui vient après
        "when": ["date"],
        "how":["Resource"],
        "in": ["Place"],
        "which": ["Resource"]  # Prendre le NP qui vient après
    }
  

  if(dico.get(res, [""]) == [""]):
    return ["Resource"]
  else: 
     return dico.get(res, [""])

switch_question_type("Give me the homepage of Forbes")


['Forbes']


['Resource']

In [ ]:
nltk.download('maxent_ne_chunker')
nltk.download('words')

[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Package words is already up-to-date!


True

Subject est la fonction qui me permet de recuperer les sujets de la question, les mots clé uniquement mais sans l'entité de la question.

In [ ]:
def subject(question,entite):
  
  ex = nltk.word_tokenize(question)
  remove_stop_word(ex)
  res = remove_stop_word(ex)  #on retire les stop word
  #print(res)
  for i in remove_stop_word(ex): #on parcours afin de retirer l'entite
    if(len(entite)>=1):
      
      for j in entite:
      
        if i == j:
          res.remove(j)
        if(" " in j):
          
          for y in j.split():
             if i == y:
               res.remove(y)
           
        
    

  return res[1:]

subject("Give me the homepage of Forbes.",get_named_entity("Give me the homepage of Forbes."))

['homepage']

sparqlRequest est la fonction qui permet de creer la requete sparql en ayant en parametre l'entite et le sujet concerné


In [ ]:
def sparqlRequest(entite,data): # entite est l'entite de la question et data correspond au mot cle qui l'accompagne
  
  sparql = SPARQLWrapper("http://dbpedia.org/sparql")
  listRes = []
  a = """
  PREFIX dbo: <http://dbpedia.org/ontology/>
  PREFIX res: <http://dbpedia.org/resource/>
  SELECT DISTINCT ?uri WHERE {
  res:"""+ entite +""" """+data+""" ?uri .}"""
  
  sparql.setQuery(a)
  sparql.setReturnFormat(JSON)
  qres = sparql.query().convert()
  try :
    if(len(qres["results"]["bindings"]) is not 0):  #je parcours l'arbre xml pour ne recuperer que les reponses
      for i in qres["results"]["bindings"]:
        
        listRes.append(i["uri"]['value'])
      return listRes
    else:
      return []
    
      
  except :
     print("UH OH ERROR")
     return -1


ouverture du fichier en mode lecture. 
Je fais un copy du contenu mais sans les 4 premiers caracteres afin de ne pas recuperer le dbo: ou le dbp: a chaque debut de mot.

In [ ]:
f = open("/content/relations.txt", "r")
data = f.read().splitlines()
rawD = data.copy()

copyWithout = data
for i in range(len(data)):
  copyWithout[i] = data[i][4:]


la fonction synonyme me permet avec la liste de mot qui correspond au mot sans les entites et sans les stop word de voir s'il y a une similarite entre les mots dans la liste et les mots dans le fichier relation

In [ ]:
def synonym(listMot):
  max=0
  motMax=""
  tokenMax=""
  relationss = copyWithout
  for token in listMot:
    try:
      synset1 = wn.synset(token+'.n.01')
      for mot in range(len(relationss)):
        try:
          synset2 = wn.synset(relationss[mot]+'.n.01')
          if(wn.path_similarity(synset1, synset2) >0.125 and wn.path_similarity(synset1, synset2) >max):
            max= wn.path_similarity(synset1, synset2)
            motMax= copyWithout[mot]
            indexMax = mot
            tokenMax=token
        except (Exception):
          pass
    except (Exception):
      pass
     
  if motMax =="":
    return False
  
  return indexMax


synonym("created")

False

In [ ]:
pip install fuzzywuzzy

la fonction lemmer utilise la librairie fuzzy qui va nous aider a avoir un valeur entre 1-100. plus la valeur est proche de 100, plus les 2 mots sont proches. Jai choisi comme limite la valeur 50.

In [ ]:
from fuzzywuzzy import fuzz

def lemmer(a,b):
  similarity = fuzz.ratio(a,b)
  return similarity>50

lemmer('creator',"created")


True

la fonction wordInWord permet de savoir si un mot est compris dans un autre.

In [ ]:
def wordInWord(w1,w2):
  w1 = w1.lower()
  w2= w2.lower()
  if(len(w1)>len(w2)):
    tmp = w1
    w1 = w2
    w2 = tmp
  if( w2.find(w1) != -1):
    return True
  else:
    return lemmer(w1,w2)



wordInWord("source2Location","location")

True

La methode responseLookupParser me permet de parcourir l'arbre xml recu en faisant la requete lookup dbpedia.



In [ ]:
import xml.etree.ElementTree as ET
def responseLookupParser(res):
 
  tree = ET.ElementTree(ET.fromstring(res))
 
  root = tree.getroot()
  uri = []
  for child in root:
    for data in child:
        if data.tag == "URI":
          uri.append(data.text)
  return uri
        
        


htmlParser est la fonction qui va recuperer les donnees qu'on recupere en faisant une requete get sur l'url dbpedia choisi. En parcourant les donnees et a l'aide d'un regex on essaye de recuperer la bonne reponse.

In [ ]:
def htmlParser(data,sujet ):  
  print(sujet)
  
  res = []
  alldata = data.splitlines()
  
  for j in alldata:
    for i in j.split():
      x = re.findall(r"db[a-z]+:[A-z]*"+sujet[0]+"[A-z]*", i)
      if(x != []):
        res.append(x)
  return res

In [ ]:
def similarite(word1,word2,sujet=""):
    
    if(len(word1)>len(word2)):
      tmp = word1
      word1 = word2
      word2 = tmp
    if(wordInWord(word1, word2)):
      
      return True
    elif synonym(sujet):
      
      return rawD[synonym(sujet)]
    
    return lemmer(word1, word2)
     
similarite("source2Location","location")

True

In [ ]:
from IPython.display import clear_output 


def makeLink(question):
  
  print("LA QUESTION = >>",question)
  entity = get_named_entity(question)
  resList = []
  rwd = rawD
  dboCorrect = []
  s = subject(question,entity)
  if(s == []):
    s = switch_question_type(question)
  print('Entite: ',entity)
  print('Les sujets: ',s)

  for i in s:
    for j in range(len(copyWithout)):

      if(similarite(copyWithout[j],i,s)):
        if( type(similarite(copyWithout[j],i,s)) is str):
          dboCorrect.append(similarite(copyWithout[j],i,s))
          break
        else:
          dboCorrect.append(rwd[j])
          
    
    dboCorrect = list(set(dboCorrect))
    if(len(dboCorrect)>1):
      for  spark in dboCorrect:
        for p in entity:
          
          a = sparqlRequest(p.replace(" ", "_"),spark)
          if( a != []):
            if(len(a)>11):
              resList.append(a[:10])
            else:
              resList.append(a)
              break
      

  
  
  if(resList == []):
    best = []
    typo = switch_question_type(question)
   
    
    for i in entity:
       
       #lookup="https://lookup.dbpedia.org/api/search?maxResults=10&query="+i.replace(" ", "%20")
       lookup =  "http://lookup.dbpedia.org/api/search/KeywordSearch?QueryClass={0}&QueryString={1}".format(typo[0], i.replace(" ", "%20"))
       print(lookup)
       try:
         response = requests.get(lookup)
         best.append(getBestDbo(responseLookupParser(response.text),i))
         print(best)
         for i in best:
          res2 = requests.get(i)
      
         if(len(dboCorrect)==0):
          return []
    
    
    
         t = [y for x in [typo, dboCorrect] for y in x]

    
         dboLookup = htmlParser(res2.text,t)
    
         for i in s:
            dboLookup.append([i])
    
    
         goodDbo = []     
         for i in dboLookup:
      
             for j in range(len(copyWithout)):
        
                if(wordInWord(copyWithout[j],i[0])):
                    goodDbo.append(rawD[j])
    
         a = []
         for i in goodDbo:
            a = sparqlRequest(lookupEntity(best),i)
            if( a != []):
                resList.append(a)
            
         return resList
       except (requests.exceptions.SSLError):
            print('SSLCertVerificationError')
            return -1
  else:
      return resList

   
     
    
    

getBestDbo() permet de choisir parmis les different url recuperer grace a lookup le meilleur url dans notre cas, en comparant l'entite de l'url et l'entite de la question.


In [ ]:
def getBestDbo(listo,entite):  #listo est la liste des url et entite est l'entite de la question
  
  for i in listo:
    if(lookupEntity(i) == entite[0]):
      
      return i
  return listo[0]


lookupEntity() permet de recuperer l'entite d'une url de type:http://dbpedia.org/resource/Canada en la parsant


In [ ]:
def lookupEntity(data):
  s = data[::-1]
  res = []
  for i in s:
    if i != '/':
      res.append(i)
    else: 
      fin = ''.join(res)
      return fin[::-1]
lookupEntity('http://dbpedia.org/resource/Canada')



'Canada'

Si vous souhaitez generer une question aleatoire et tester le programme

In [ ]:
questionTest = questions[random.randint(0,len(questions)-1)]
print(questionTest)
makeLink(questionTest)


Which states border Illinois?
LA QUESTION = >> Which states border Illinois?
Entite:  ['Illinois']
Les sujets:  ['states', 'border']
http://lookup.dbpedia.org/api/search/KeywordSearch?QueryClass=Resource&QueryString=Illinois
SSLCertVerificationError


-1

Tester() est la fonction qui va tester la reponse generee par mon programme et la vrai reponse

In [ ]:
def tester(i):
  print('QUESTION: ',questionsTest[i])
  print('REPONSE ATTENDU',answersTest[i])

  a = makeLink(questionsTest[i])
  print(a)
  if(a == -1): 
    return -1
  
  if(a != []):
    for i in answersTest[i]:
      
      if(lookupEntity(i) == lookupEntity(a[0][0])):
        print("BRAVO")
        return True
  
  print("Mince cest ratee")
  return False



In [ ]:
def final():
  print("----------------------------------\nRealisé par KAROUIA ALAEDINE MIAGE M2 IA\n----------------------------------\n")
  res = 0
  cancel = 0
  for i in range(len(questionsTest)):
    if(tester(i) == -1):
      print('cancel')
      cancel += 1

    if(tester(i) == True):
      res +=1
  
  print("recall: {} %".format((res/(len(answersTest) - cancel))*100)) #moyenne des reponse en enlevant quand le lookup fait une erreur
  print("precision {} %".format((res/len(answersTest))*100)) #moyenne des bonnes reponses
  #le f globale
  print("mesure F globale {} ".format((2* (res/len(answersTest) * (res/len(answersTest) - cancel ))/((res/len(answersTest) + (res/len(answersTest) - cancel )) *100))))
final()


----------------------------------
Realisé par KAROUIA ALAEDINE MIAGE M2 IA
----------------------------------

QUESTION:  Which river does the Brooklyn Bridge cross?
REPONSE ATTENDU ['http://dbpedia.org/resource/East_River']
LA QUESTION = >> Which river does the Brooklyn Bridge cross?
Entite:  ['Brooklyn Bridge']
Les sujets:  ['river', 'cross']
[['http://dbpedia.org/resource/East_River']]
BRAVO
QUESTION:  Which river does the Brooklyn Bridge cross?
REPONSE ATTENDU ['http://dbpedia.org/resource/East_River']
LA QUESTION = >> Which river does the Brooklyn Bridge cross?
Entite:  ['Brooklyn Bridge']
Les sujets:  ['river', 'cross']
[['http://dbpedia.org/resource/East_River']]
BRAVO
QUESTION:  Who created Wikipedia?
REPONSE ATTENDU ['http://dbpedia.org/resource/Jimmy_Wales', 'http://dbpedia.org/resource/Larry_Sanger']
LA QUESTION = >> Who created Wikipedia?
Entite:  ['Wikipedia']
Les sujets:  ['created']
[['2013-10-23', '2015-03-13', 'July 2021']]
Mince cest ratee
QUESTION:  Who created Wiki